In [1]:
import tensorflow as tf
import numpy as np

In [2]:
data=np.load("C:\\MLDatabases\\google-images-download-master\\downloads\\100x100_resized_images_as_numpy\\x_non_alpha_composed.npy",mmap_mode='r')

In [3]:
labels=np.load("C:\\MLDatabases\\google-images-download-master\\downloads\\100x100_resized_images_as_numpy\\y_non_alpha_composite1.npy",mmap_mode='r')

In [4]:
def create_weights(shape,name,stddev=0.003):
    return tf.Variable(initial_value=tf.truncated_normal(shape=shape, mean=0, stddev=stddev, dtype="float32", seed=10),dtype="float32",name=name)

In [5]:
tf.set_random_seed(10)
np.random.seed(10)

In [ ]:
tf.reset_default_graph()
def get_graph():
    
    strides=(1,1,1,1)    
    
    with tf.device("/device:CPU:0"):
    
        x=tf.placeholder(tf.float32,shape=(None,100,100,3),name="x")
        y=tf.placeholder(tf.int64,shape=(None,),name="y")
        lr=tf.placeholder(tf.float32,shape=(),name="lr")
        dr=tf.placeholder(tf.float32,shape=(),name="dr")
        #y_onehot=tf.one_hot(y,2)


        conv_w1=create_weights((3,3,3,2),"conv_w1")
        conv_w2=create_weights((3,3,2,2),"conv_w2")
        conv_w3=create_weights((3,3,4,2),"conv_w3")
        conv_w4=create_weights((3,3,8,2),"conv_w4")
        conv_w4=create_weights((3,3,8,4),"conv_w5")
        conv_w4=create_weights((3,3,8,4),"conv_w6")
        conv_w4=create_weights((3,3,8,4),"conv_w7")
        conv_w4=create_weights((3,3,8,4),"conv_w8")
        conv_w4=create_weights((3,3,8,4),"conv_w9")
        conv_w4=create_weights((3,3,8,8),"conv_w10")
        conv_w4=create_weights((3,3,8,8),"conv_w11")
        conv_w4=create_weights((3,3,8,8),"conv_w12")
        conv_w4=create_weights((3,3,8,8),"conv_w13")
        conv_w4=create_weights((3,3,8,8),"conv_w14")
        #conv1_mean=create_weights((16,),"conv1_mean",1)
        #conv2_mean=create_weights((16,),"conv2_mean",1)
        #conv3_mean=create_weights((16,),"conv3_mean",1)
        #conv4_mean=create_weights((16,),"conv4_mean",1)
        #conv1_variance=create_weights((16,),"conv1_variance",1)
        #conv2_variance=create_weights((16,),"conv2_variance",1)
        #conv3_variance=create_weights((16,),"conv3_variance",1)
        #conv4_variance=create_weights((16,),"conv4_variance",1)

        w_fc1=create_weights((7*7*8,512),"w_fc1")
        fc1_bias=create_weights((512,),"fc1_bias")
        w_fc2=create_weights((512,2),"w_fc2")
        fc2_bias=create_weights((2,),"fc2_bias")



        conv1=tf.nn.conv2d(input=x,filter=conv_w1,strides=strides,padding="SAME",name="conv1") #conv1=45
        conv1_relu=tf.nn.relu(conv1,name="relu_conv1")
        conv1_bn=tf.layers.batch_normalization(conv1_relu,training=True)
        #conv1_bn=tf.nn.batch_normalization(x=conv1_relu,mean=conv1_mean,variance=conv1_variance,variance_epsilon=0.000004,offset=None,scale=None)


        conv2=tf.nn.conv2d(input=conv1_bn,filter=conv_w2,strides=strides,padding="SAME",name="conv2") #conv2=21
        conv2_relu=tf.nn.relu(conv2,name="relu_conv2")
        conv2_bn=tf.layers.batch_normalization(conv2_relu,training=True)
        #conv2_bn=tf.nn.batch_normalization(x=conv2_relu,mean=conv2_mean,variance=conv2_variance,variance_epsilon=0.000004,offset=None,scale=None)


        conv3=tf.nn.conv2d(input=conv2_bn,filter=conv_w3,strides=strides,padding="SAME",name="conv3") #conv3=10
        conv3_relu=tf.nn.relu(conv3,name="relu_conv3")
        conv3_bn=tf.layers.batch_normalization(conv3_relu,training=True)
        #conv3_bn=tf.nn.batch_normalization(x=conv3_relu,mean=conv3_mean,variance=conv3_variance,variance_epsilon=0.000004,offset=None,scale=None)


        conv4=tf.nn.conv2d(input=conv3_bn,filter=conv_w4,strides=strides,padding="SAME",name="conv4") #conv4=5
        conv4_relu=tf.nn.relu(conv4,name="relu_conv4")
        conv4_bn=tf.layers.batch_normalization(conv4_relu,training=True)

        #conv4_bn=tf.nn.batch_normalization(x=conv4_relu,mean=conv4_mean,variance=conv4_variance,variance_epsilon=0.000004,offset=None,scale=None)
        flat_feature=tf.reshape(tensor=conv4_bn,shape=[-1,7*7*8],name="flat_feature")
        print(flat_feature)
        fc1=tf.add(tf.matmul(flat_feature,w_fc1),fc1_bias,name="fc1")
        fc1_dropout=tf.nn.dropout(fc1,keep_prob=dr)

        logits=tf.add(tf.matmul(fc1_dropout,w_fc2),fc2_bias,name="fc2")

        with tf.device("/device:CPU:0"):
            softmaxed_logits=tf.nn.softmax(logits,name="softmaxed_logits")    
        loss=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(None,labels=y, logits=logits),name="loss")

        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        with tf.control_dependencies(update_ops):
            optimizer=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

        accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(softmaxed_logits,axis=1), y),dtype=tf.float32))
    
    return optimizer,loss,accuracy,x,y,lr,dr
    
    
    
    